This model tries to optimize stress-strain state (reduce displacements to acceptable level) by combination of simple finite element analysis and reinforcement learning.

In [1]:
import numpy as np
import gym
from gym import spaces
import time
import random
from stable_baselines3 import A2C
import torch
from stable_baselines3.common.env_checker import check_env

In [2]:
torch.__version__

'1.13.1'

In [3]:
gym.__version__

'0.21.0'

# Finite Element Model of Axially Loaded Bar

In [4]:
# Details of model can be found at:
# https://en.wikiversity.org/wiki/Introduction_to_finite_elements/Axial_bar_finite_element_solution

In [5]:
def elementStiffness(A, E, h):
    s= A*E/h
    return s*np.array([[1,-1],[-1,1]])

In [6]:
def elementLoad(node1, node2, a, h):

    x1 = node1
    x2 = node2

    fe1 = a*x2/(2*h)*(x2**2-x1**2) - a/(3*h)*(x2**3-x1**3)
    fe2 = -a*x1/(2*h)*(x2**2-x1**2) + a/(3*h)*(x2**3-x1**3)
    return np.array([fe1,fe2])

In [7]:
def AxialBarFEM(A,E):
    L = 1.0
    a = 1.0
    R = 1.0    
    e = 3
    h = L/e
    n = e+1
        
    node=[]    
    for i in range(n):
        node.append(i*h)
    node=np.array(node) 
        
    elem=[]    
    for i in range(e):
        P=[i,i+1]
        elem.append(P)
    elem=np.array(elem)    
      
    K=np.zeros((n,n))   
    F=np.zeros((n,1))  
       
    for i in range(e):
        node1 = elem[i,0]
        node2 = elem[i,1]
        Ke = elementStiffness(A, E, h)
        fe = elementLoad(node[node1],node[node2], a, h)
        K[node1:node2+1,node1:node2+1] = K[node1:node2+1,node1:node2+1] + Ke
        F[node1:node2+1] = F[node1:node2+1] + fe.reshape(2,1)
         
    F[n-1] = F[n-1] + 1.0
   
    bc_node=[0]
    bc_val=[0]
    # https://github.com/CALFEM/calfem-matlab/blob/master/fem/solveq.m
    
    
    bc=np.array([bc_node, 
                bc_val]).T
    nd, nd=K.shape
    fdof=np.array([i for i in range(nd)]).T
    d=np.zeros(shape=(len(fdof),))
    Q=np.zeros(shape=(len(fdof),))

    pdof=bc[:,0].astype(int)
    dp=bc[:,1]
    fdof=np.delete(fdof, pdof, 0)
    s=np.linalg.lstsq(K[fdof,:][:,fdof], (F[fdof].T-np.dot(K[fdof,:][:,pdof],dp.T)).T, rcond=None)[0] 
    d[pdof]=dp
    d[fdof]=s.reshape(-1,)
    
#     Q=np.dot(K,d).T-F 
    return d[-1],A,E

In [8]:
# Input: cross-sectional area and Young's modulus
# Output: largest displacement at rightmost node at the point of external force application

DIM=len(AxialBarFEM(0.5, 1))

## Utils

In [9]:
def prestep(action,A,E):
    d=0.01
    d1=0.005
    if action==0:
        return A-d, E
    elif action==1:
        return A-d,E-d1
    elif action==2:
        return A-d,E+d1
    elif action==3:
        return A,E
    elif action==4:
        return A+d,E
    elif action==5:
        return A+d, E+d1
    elif action==6:
        return A+d, E-d1
    elif action==7:
        return A, E+d1
    else:
        return A,E-d1

In [10]:
def reward_(obs_,obs):
#     if obs_[1]>obs[1]: # use when minimizing cross-sectional area
    if obs_[0]>obs[0]:  # use when minimizing displacement  
        return 1
    else:
        return 0

## Reinforcement learning model

In [11]:
N_DISCRETE_ACTIONS=8

In [12]:
class BarEnv(gym.Env):
    
    metadata = {"render.modes": ["human"]}

    def __init__(self):
        super().__init__()
        self.action_space = spaces.Discrete(N_DISCRETE_ACTIONS)
        
        self.A=3*random.random()
        self.E=2*random.random()
        self.obs=AxialBarFEM(self.A, self.E)   
        self.observation_space = spaces.Box(low=np.array([-np.inf for x in range(DIM)]),
                                            high=np.array([np.inf for y in range(DIM)]),
                                            shape=(DIM,),
                                           dtype=np.float64)
        self.needs_reset = True

    def step(self, action):
        
        obs_=self.obs
        self.A,self.E=prestep(action,self.A,self.E)
        self.obs=AxialBarFEM(self.A,self.E)
        reward=reward_(obs_,self.obs)
                
        done=False
        if self.obs[0]<0.1: 
            done = True
        
        if self.needs_reset:
            raise RuntimeError("Tried to step environment that needs reset")
            
        if done:
            self.needs_reset = True
      
        return np.array(self.obs), reward, done, dict()

    def reset(self):
        self.A=3*random.random()
        self.E=2*random.random()
        self.obs=AxialBarFEM(self.A, self.E) 
        self.needs_reset = False
        return np.array(self.obs)  

    def render(self, mode="human"):
        pass    

    def close(self):
        pass

In [13]:
env = BarEnv()
check_env(env)

In [14]:
start=time.time()
model = A2C("MlpPolicy", env).learn(total_timesteps=1_500_000)
end=time.time()   

In [15]:
print('Total time taken: {} min'.format((end - start)/60))

Total time taken: 68.04455128510793 min


### AI design

In [16]:
obs = env.reset()

In [17]:
env.A=0.7
env.E=0.9

In [18]:
THR=2

In [19]:
i=0
while i<100:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    if obs[0]<THR and obs[0]>0.8*THR: # use when minimizing displacement
        break
#     if obs[0]<THR or obs[0]>3*THR : # use when minimizing cross-sectional area
#         break  
    i+=1   

In [20]:
if obs[0]>THR:
    print("Bad initial parameters! Try increasing initial cross-sectional area A, Young's modulus E and/or number of iterations")
elif obs[0]<0.8*THR:
    print("You can get better parameters. Try decreasing initial area A and/or Young's modulus E")
else:    
    print("Solution converged! MaxDispl={}, A={},E={}".format(obs[0],obs[1],obs[2]))

Solution converged! MaxDispl=1.9753086419753065, A=0.75,E=0.9
